In [1]:
import time
import signal

import numpy as np
from pynq import Overlay
from isa import REGMAP, lui, load_instructs

In [2]:
instructs = load_instructs("main.hex")
variables = [
    2,  # number of row
    2,  # number of column
    0,  # cost[0][0]
    1,  # cost[0][1]
    2,  # cost[1][0]
    3,  # cost[1][1]
]
start_sp = lui(2, 0xA0030000)
instructs.insert(0, start_sp)

In [5]:
t0 = time.perf_counter()

overlay = Overlay("./bitstream/riscv.bit")
imem = overlay.IMEM_CONTROL
dmem = overlay.DMEM_CONTROL
gpio = overlay.axi_gpio_0

for idx, i in enumerate(instructs):
    imem.write(idx * 4, i)
for idx, v in enumerate(variables):
    dmem.write(idx * 4, v)
gpio.write(0x00, 1)
gpio.write(0x00, 0)

results = []
for i in range(20):
    result = dmem.read((i) * 4)
    results.append(result)
    print(f"{i}:   {result}")

# results[6] is not use just for checking.
output = np.array([[results[7], results[9]], [results[8], results[10]]], dtype=np.int32)
print(f"Result 2x2  of Hungarian algorithm: \n {output}")
t1 = time.perf_counter()
print(f"All processes are done in {t1 - t0} seconds")

0:   2
1:   2
2:   0
3:   1
4:   2
5:   3
6:   6
7:   0
8:   0
9:   1
10:   1
11:   0
12:   0
13:   0
14:   0
15:   0
16:   0
17:   0
18:   0
19:   0
Result 2x2  of Hungarian algorithm: 
 [[0 1]
 [0 1]]
All processes are done in 0.677481650000118 seconds
